In [ ]:
import os

os.system('pip install -r requirements.txt')

In [ ]:
import torch
import torchvision

print(torch.__version__)
print(torchvision.__version__)

if torch.backends.mps.is_available():
    device = 'mps'
elif torch.cuda.is_available():
    device = 'cuda'
else:
    'cpu'
print(f"Using device: {device}")

class_names = ['adj', 'int', 'geo', 'pro', 'non']

In [ ]:
from torchvision.models.detection import fasterrcnn_resnet50_fpn_v2, FasterRCNN_ResNet50_FPN_V2_Weights

weights = FasterRCNN_ResNet50_FPN_V2_Weights.DEFAULT
model = fasterrcnn_resnet50_fpn_v2(weights=weights)

num_classes = len(class_names)
in_features = model.roi_heads.box_predictor.cls_score.in_features
model.roi_heads.box_predictor = torchvision.models.detection.faster_rcnn.FastRCNNPredictor(in_features, num_classes)

model.load_state_dict(torch.load('fasterrcnn_resnet50_fpn.pth', map_location=device))

In [ ]:
from torchvision.io.image import read_image
from torchvision.utils import draw_bounding_boxes
from torchvision.transforms.functional import to_pil_image
from torchvision.transforms import v2 as T


trans = T.Compose([
            T.ToImage(),
            T.Resize((600, 600)),
            T.ToDtype(torch.float32, scale=True),
            T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
        ])

img = read_image("path/to/photo")
wigth, height = img.size(1), img.size(2)
img = trans(img.to(device))

model.eval()
with torch.inference_mode():
    prediction = model(img.unsqueeze(0))[0]

labels = [class_names[i] for i in prediction["labels"]]
img = (255.0 * (img - img.min()) / (img.max() - img.min())).to(torch.uint8).to('cpu')
img = img[:3, ...]
box = draw_bounding_boxes(img, boxes=prediction["boxes"],
                          labels=labels,
                          colors="red",
                          width=4)
im = to_pil_image(box.detach())
im